In [2]:


import os
import pandas
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_together import ChatTogether
import pandas as pd
from google.cloud import bigquery
from google.generativeai import caching
import google.generativeai as genai
from  datetime import timedelta
import time 
import os
import time
import shutil
from pathlib import Path
from datetime import datetime
import langgraph
from langgraph.graph import Graph
from langgraph.graph import END
import streamlit as st 
from google.cloud import bigquery
import base64
import json
import google.auth
from google.oauth2 import service_account
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools import tool
#import propmt template,agent executor,react agent
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

from langchain_anthropic import ChatAnthropic
from langchain_groq import ChatGroq
import requests 
import re
import ast

# setting env variables
credentials_b64 = os.environ['credentials_b64']

credentials_bytes = base64.b64decode(credentials_b64)
credentials_dict = json.loads(credentials_bytes)
credentials = service_account.Credentials.from_service_account_info(credentials_dict)




llm=ChatOpenAI(model='gpt-4o-mini')
llm2=ChatOpenAI(model='gpt-4o-mini')
# llm=ChatGroq(model='llama-3.1-70b-versatile')
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain2=prompt | llm2
with open("./schema.txt", "r") as file:
    schema=file.read()



history =None
history2=None

chat =None 
chat2=None
n=0

def find_references(user_query,model='gpt-4o-mini',stream=True):
    global history,chat,chat2,history2
    global n
    n=0
    history=ChatMessageHistory()
    chat = RunnableWithMessageHistory(
        chain,
        lambda session_id: history,
        input_messages_key="input",
        history_messages_key="chat_history",
    )
    history2=ChatMessageHistory()
    chat2 = RunnableWithMessageHistory(
        chain2,
        lambda session_id: history2,
        input_messages_key="input",
        history_messages_key="chat_history",
    )

    res_gem=user_query
    res_gem+='\n'
    ti=time.time()
    if model=='sonnet':
        llm = ChatAnthropic(temperature=0, model_name="claude-3-sonnet-20240229")
    elif model =='gpt-4o-mini':
        llm = ChatOpenAI(model="gpt-4o-mini")
    # elif model=='gemini':
    #     llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
    else:
        llm = ChatGroq(model='llama-3.1-70b-versatile')  
  
    @tool
    def get_value_from_column(list1:str) -> str:
        """
        This function retrieves the specific value present in database column_name that is most similar to the search_value.
        input: 2D list of lists  with the format shown below.
        ([[column_name1,search_value1], [column_name2,search_value2], [column_name3,search_value3]]

        """
        def get_value_from_column1(list1:list) -> str:



            column_name=list1[0]
            search_value=list1[1]
            if column_name not in ['venue', 'series_name',  'tournament_name',   'match_type',  'team_bat',  'team_bowl',  
            'bowler_type', 'batter','bowler', 'bowler_kind','batter_hand']:
                return """Can only  search for this columns ['venue', 'series_name',  'tournament_name',   'match_type',  'team_bat',  'team_bowl',  
            'bowler_type', 'batter','bowler', 'bowler_kind','batter_hand']  in database"""
            if column_name=='batter' or column_name=='bowler':
                API_KEY = os.environ['BING_API_KEY']
                URL = "https://api.bing.microsoft.com/v7.0/search"

                HEADERS = {"Ocp-Apim-Subscription-Key": API_KEY}
                PARAMS = {
                    "q": search_value + ' cricinfo ',
                    "count": 10,
                    "offset": 0,
                    "mkt": "en-US",
                    "safeSearch": "Moderate"
                }

                response = requests.get(URL, headers=HEADERS, params=PARAMS)
                results = response.json()
                flag = False

                for result in results["webPages"]["value"]:
                    url = result["url"]
                    if 'https://www.espncricinfo.com/cricketers/' in url:
                        flag = True
                        break

                if flag is False:
                    return "Player not found Leave this column's value  empty."

                parts = url.split("-")
                id = parts[-1]
                res = requests.get('http://core.espnuk.org/v2/sports/cricket/athletes/' + str(id))
                name = res.json()['displayName']
                return str(name)
            else:

            # Load FAISS index for specified column
                embeddings = OpenAIEmbeddings(
                    model='text-embedding-ada-002'
                )
                db = FAISS.load_local(fr"C:\Users\adith\Documents\Projects\python-projects\CRICAI-SQL\vector_databases\{column_name}", embeddings, allow_dangerous_deserialization=True)

                # Create document retriever and find most relevant document to player name
                retriever = db.as_retriever(search_type='mmr', search_kwargs={'k': 5, 'lambda_mult': 1})
                matched_value = retriever.get_relevant_documents(search_value)
                res=[]
                for i in matched_value:
                    res.append(i.page_content)
                return str(res)

        # print(type(list1))
        # print(list1)


        list_str = re.search(r'\[.*\]', list1)
        if list_str:
            list_str = list_str.group()
        else:
            return "Invalid input format: No list found"
        list1 = ast.literal_eval(list_str)
        # list1 = eval(list_str)
        # list1=eval(list1)

        # print(type(list1))
        # print(list1)
        l=[]
        for list2 in list1:
            if len(list2)!=2:
                print("list2",list2)
                return "Inputs are not in correct format."
                
            res=get_value_from_column1(list2)
            l.append(list2+[res])
        print(l)
        return l 
        
    @tool
    def add_2_numbers(a,b) :
        """ 
        This function adds 2 numbers.
      """
        return a+b
        

    tools = [get_value_from_column,add_2_numbers]
    system_prompt=""" 
   You are an helpful assistant.

    """
    user_prompt=f"""   
    User Query: "{user_query}"
    -
    Finally you need to return the final answer with correct column names and values that are present in the database.
    Thats why,you need  to preprocess these user queries by seraching in the database using the tool get_value_from_column
    and then return the final answer.

    


    Some of the important columns,and its values in the database are:
    - venue : many values are present in this column, so you may need to search in database using a function that is mentioned below.
    - series_name : many values are present in this column, so you may need to search in database using a function that is mentioned below.
    - tournament_name : many values are present in this column, so you may need to search in database using a function that is mentioned below.
    - team_bat : many values are present in this column, so you may need to search in database using a function that is mentioned below.
    - team_bowl : many values are present in this column, so you may need to search in database using a function that is mentioned below.
    - batter : many values are present in this column, so you may need to search in database using a function that is mentioned below.
    - bowler : many values are present in this column, so you may need to search in database using a function that is mentioned below.
    - bowler_kind: ['pace bowler','spin bowler'] (you may need not search in database for this columns,values as they are only few)
    - batter_hand :['Right Hand Batter','Left Hand Batter'] (you may need not search in database for this columns,values as they are only few)
    - match_type : ['ODI','MDM','Test','IT20','T20','ODM'] (you may need not search in database for this columns,values as they are only few)

    - bowler_type : ['RAP'(abbreviated for right arm pace),'OB'(abbreviated for off break),'LWS'(abbreviated for left wrist spin),
    'LAP'(abbreviated for left arm pace),'LB'(abbreviated for left arm break),'SLA'(abbreviated for slow left arm)
    ] (you may need not search in database for this columns,values as they are only few)

    -another point is that tournament_name consists of  name of series,event..
    but the seriesname consists of tournamanet_name+Season  of the series/event..
      for example:tournament_namecan be :IPL ,series_name:IPL 2024 season: 2024
    thus for query involving just the name of event you can use directly the tournament_name,if specifically
    mentioned the event/tournament with a season/year you can use series_name.







    """
    MEMORY_KEY = "chat_history"
    # prompt for tool calling agent
    # prompt = ChatPromptTemplate.from_messages([
    #     ("system", "you're a helpful assistant"), 
    #     ("human", "{input}"), 
    #     ("placeholder", "{agent_scratchpad}"),
    # ])  

    prompt = PromptTemplate.from_template(
    """
  
    You are an intelligent agent designed to assist in generating SQL queries(specifically filtering dataset i.e in where clause) by processing  based on user query.


    You will be given a user query,in which users may provide vague references to  names, such as batter,bowler,bowler_type,
    batter_hand,bowler_kind,match_type,team_bat,team_bowl,tournament_name,series_name,venue in a cricket database,
    but directly using those names/values of that column for filtering the dataset in sql may not work always.
    -so you need to search for the exact name of that column in the database using the tool get_value_from_column ( except for bowler_kind,batter_hand,match_type which have only few values and i have provided below  you with the list of those values).


    Thus you need  to preprocess these user queries by searching in the database using the tool get_value_from_column and 
    then return the final answer.


    Remember:
    -Even if in the query users mentioned other columns other than the ones mentioned above,you need to ignore them.    




    Important important columns,and its values in the database are:
    columns that has many distinct values are:
    - venue 
    - series_name
    - tournament_name
    - batter
    - bowler
    - team_bat
    - team_bowl

    you may need to search for values in above columns in database.
    info about tournament_name and series_name:

        -tournament_name column  mentions the  general name of series/event but the series_name column
        specifies both the series/event with a season/year (i.e the series_name=tournament_name +season/year).  
        -thus for queries involving just the name of event  can use directly the tournament_name,if specifically
        mentioned the event/tournament with a season/year  can use series_name.




    columns that has few distinct values and its values are known to us are:
    - bowler_kind: ['pace bowler','spin bowler'] 
    - batter_hand :['Right Hand Batter','Left Hand Batter'] 
    - match_type : ['ODI','MDM','Test','IT20','T20','ODM'] 
    - bowler_type : ['RAP'(means  right arm pace),'OB'(means off break),'LWS'(means left wrist spin),
    'LAP'(means left arm pace),'LB'(means left arm break),'SLA'(means slow left arm)
    ] 
    you many need not to search in database for these columns as their values are known to us.




    ---------------------
    You have access to the following tools:

    {tools}
    ---
    Tool names:
    {tool_names}
    ---------------------------------------------------------------------------------------------------

    Use the following format:

    User Query: {input}

    Thought: Analyze the user query and determine what information needs to be retrieved from the database. 
    Consider which columns are relevant and what search terms to use.

    Action: get_value_from_column

    Action Input: [['column_name1', 'search_value1'],['column_name2', 'search_value2'], ...]

    Observation: result of action.


    (Repeat the Thought/Action/Action Input/Observation steps N times, but dont repeat the same action again and again)

    After you have done the above steps N times,

    -understand/reason which values found in database are most relevant and accurate for filtering the dataset for user query. 



    Thought: Analyze the observations from all actions. 


    Final Answer: Provide  Final Answer that includes
    -The relevant column names,appropriate values based on results of actions found for each column. from the database.



    -But Dont include  sql query.



    Begin!

    User Query: {input}
    Thought: {agent_scratchpad}


    """
    )

    chat_history = []
    # prompt = hub.pull("hwchase17/react")
    agent = create_react_agent(llm, tools, prompt)    
    # agent = create_tool_calling_agent(llm, tools, prompt)

    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)


    result=agent_executor.invoke({"input": user_query})

        # n=404
        # return {"response":str(e),"query":user_query,"f":None}

    print("Time Taken for gpt-4o-mini = ",time.time()-ti)
    print("response of Agent :  ",result['output'])

    while True:

        human_references=input("Your preferred refernces for filtering dataset in sql query: ")
        if human_references!='':
            res_gem+=human_references
            break 
        elif human_references=='retry':
            result=agent_executor.invoke({"input": human_references})
        elif human_references=='exit':
            exit()
        else:
            res_gem+=result['output']
    # print("Response: ",res_gem)
    
    res_gem+='\n\n'
    return {'response':res_gem,'query':user_query,'f':None}

with open("sample_codes.txt","r") as f:
  sample_codes=f.read()

task=""" 
I have a ball by ball  database of cricket matches named bbbdata.ballsnew_2406 in bigquery.

I want to write  SQL query for this user query:  {user_query}



Schema and info about columns of  Database:  

{schema}
Some of sample queries for calculating metrics on this database are:

{sample_codes}

-More info about exact column names and values from userquery present in Datbase: {res_gem}.
 Use only  this to filter dataset i.e in where of sql query.


Suggestions:
1.Use backslash as delimeter before ' is present in sql query.

 """
critique = """

I have a ball by ball  database of cricket matches.
My assistant had wrote this sql query for finding the answer for this user query: {user_query}.
His SQL Query: {sql_query}.

Can u check/verify step by step if he committed any logical error in writing sql query.
-verify if he has aggregated the data correctly to obtain the metrics.



For more detailed info: 
This are some of my codes which are logically correct i use : {sample_codes}.



And finally return the final sql query.


"""
def coding(json_data):
    res_gem=json_data['response']
    user_query=json_data['query']
    f=json_data['f']

    remarks=user_query
    global n,critique
    n+=1

    #if after 4 iterations no valid dataframe is obtained as result then return remarks.
    if n>7:
        remarks="Cannot be processed further. Simplify the Quey and try again."
        return [remarks,"error"]
    


    #if a valid dataframe is obtained as result then return ask user for suggestions for sql query to be improved or else return the result    
    if type(f)==pandas.core.frame.DataFrame:
        print("Result:\n")
        print(f)
        s=input("Are u satisfied with the result or want to give any suggestion for the sql query to be improved ? ")

        if s=='':
            n=10
            return [remarks,f]
        else:
            
            f=s

      
    #based on which iteration it is ask ai 
    ti=time.time()
    def get_sql_query(model_response):

        pattern = r'```sql\n(.*?)```'
        match_sql = re.findall(pattern, model_response, re.DOTALL)
        
        if match_sql:
            sql_query = match_sql[-1].strip()
            return sql_query
        else:
            return None
    # for first iteration
    if n==1:
        f_user_query=task.format(user_query=user_query,res_gem=res_gem,schema=schema,sample_codes=sample_codes)
        model_response=chat.invoke({"input": f_user_query},{"configurable": {"session_id": "unused"}}).content
        sql_query = get_sql_query(model_response)
        print(f"\n\ntime taken for sql query generation for {n}th iteration:  {time.time()-ti} \n")
        print(f"sql query generated for {n}th iteration by model  :",model_response)




        critique=critique.format(user_query=user_query,sql_query=sql_query,schema=schema,res_gem=res_gem,sample_codes=sample_codes)        
        critique_response=chat2.invoke({"input": critique},{"configurable": {"session_id": "unused"}}).content
        critique_query=get_sql_query(critique_response)
        print(f"\n\ntime taken for sql query critique for {n} th iteration {time.time()-ti} \n") 

        print(f"the critiqued response  for {n} th iteration is :",critique_response)
        time.sleep(3)
        print("\n\n")
        if critique_query:
            sql_query = critique_query


    #for 2nd to 7th iteration
      
    else:
        ti=time.time()


        failed_prompt=f"""while executing  the  sql query you previously  in bigquery,
            this error had occurred: {f}.
            Solve it..

            """
        critique_response=chat2.invoke({"input":  failed_prompt},{"configurable": {"session_id": "unused"}}).content
        critique_query=get_sql_query(critique_response) 
        if critique_query:
            sql_query = critique_query
            

    while True:
        human_suggestion_for_query=input("Any suggestion do u want to give to critiquer ? ")


        if human_suggestion_for_query=='':
            break 
        elif human_suggestion_for_query=='exit':
            exit()

        critique_response=chat2.invoke({"input":  human_suggestion_for_query},{"configurable": {"session_id": "unused"}}).content
        sql_query=get_sql_query(critique_response)    
        print(f" critique response for human in loop component : \n {critique_response}")

        time.sleep(3)
        



    return [sql_query,f]

def router(inp):
    global n
    if  n>7:
        return "exit"
    else:
        return "run"


def query_to_dataframe(queryl):
    project_id = 'adept-cosine-420005'
    query=queryl[0]
    client = bigquery.Client(project=project_id,credentials=credentials)
    try:
        query_job = client.query(query)
        query_job.result()
        df = query_job.to_dataframe()
    except Exception as e:
        return { "response": "error","query":"adi", "f": str(e) }
        # return "a","a",e
    return { "response": "success","query":query, "f": df }


#graph building


workflow = Graph()

workflow.add_node("find_references", find_references)
workflow.add_node("coding", coding)
workflow.add_node("query_to_dataframe", query_to_dataframe)

workflow.add_conditional_edges(
    "coding",
    router,
    {
        "exit": END,
        "run": "query_to_dataframe",
    },
)

workflow.add_edge('find_references', 'coding')

workflow.add_edge('query_to_dataframe', 'coding')

workflow.set_entry_point("find_references")


def ask(question):
    app = workflow.compile()
    response=app.invoke(question)
    remarks=response[0]
    df=response[1]
    if remarks=="Cannot be processed further. Simplify the Quey and try again.":
        print(remarks)
        return remarks
    else:
        #show the dataframe
        # print(df)
        return df
hardquesttions=["pooran stats against each of bowling types in ipl since 2022",
                "rohit sharma stats in icc t20 world cup season wise.",
                " compare virat kohlis stats against sla,legspin in odis for periods  between 2013-2019,2020-2024  "]
question=" srh   powerplay batting stats in ipl season wise since 2016.  "
# df=ask(question)




In [5]:
df

,season,innings,total_runs,total_balls,total_outs,Average,Strike_Rate,Dot_Percentage,hs,fifties,hundreds,total_fours,total_sixes
0,2016,17,842,714,16,52.63,117.93,42.72,74,9,0,90,22
1,2017,14,758,588,12,63.17,128.91,39.63,83,8,0,89,21
2,2018,17,856,714,26,32.92,119.89,41.46,62,8,0,107,19
3,2019,15,884,634,16,55.25,139.43,38.01,77,12,0,107,24
4,2020/21,16,853,675,18,47.39,126.37,44.15,83,10,0,98,27
5,2021,14,701,594,18,38.94,118.01,45.12,73,7,0,73,25
6,2022,14,649,588,19,34.16,110.37,47.62,61,6,0,78,16
7,2023,14,732,588,28,26.14,124.49,42.86,68,9,0,96,17
8,2024,16,1166,672,33,35.33,173.51,34.82,132,10,2,121,68


In [6]:
#create a react agent  
def find_references(user_query,model='gpt-4o-mini',stream=True):

    res_gem=user_query
    res_gem+='\n'
    ti=time.time()
    if model=='sonnet':
        llm = ChatAnthropic(temperature=0, model_name="claude-3-sonnet-20240229")
    elif model =='gpt-4o-mini':
        llm = ChatOpenAI(model="gpt-4o-mini")
    # elif model=='gemini':
    #     llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
    else:
        llm = ChatGroq(model='llama-3.1-70b-versatile')  
  
    @tool
    def get_value_from_column(list1:str) -> str:
        """
        This function retrieves the specific value present in database column_name that is most similar to the search_value.
        input: 2D list of lists  with the format shown below.
        ([[column_name1,search_value1], [column_name2,search_value2], [column_name3,search_value3]]

        """
        def get_value_from_column1(list1:list) -> str:



            column_name=list1[0]
            search_value=list1[1]
            if column_name not in ['venue', 'series_name',  'tournament_name',   'match_type',  'team_bat',  'team_bowl',  
            'bowler_type', 'batter','bowler', 'bowler_kind','batter_hand']:
                return """Can only  search for this columns ['venue', 'series_name',  'tournament_name',   'match_type',  'team_bat',  'team_bowl',  
            'bowler_type', 'batter','bowler', 'bowler_kind','batter_hand']  in database"""
            if column_name=='batter' or column_name=='bowler':
                API_KEY = "3d1d6539895b4ec18c0adc1633fce5f3"
                URL = "https://api.bing.microsoft.com/v7.0/search"

                HEADERS = {"Ocp-Apim-Subscription-Key": os.environ['BING_API_KEY']}
                PARAMS = {
                    "q": search_value + ' cricinfo ',
                    "count": 10,
                    "offset": 0,
                    "mkt": "en-US",
                    "safeSearch": "Moderate"
                }

                response = requests.get(URL, headers=HEADERS, params=PARAMS)
                results = response.json()
                flag = False

                for result in results["webPages"]["value"]:
                    url = result["url"]
                    if 'https://www.espncricinfo.com/cricketers/' in url:
                        flag = True
                        break

                if flag is False:
                    return "Player not found Leave this column's value  empty."

                parts = url.split("-")
                id = parts[-1]
                res = requests.get('http://core.espnuk.org/v2/sports/cricket/athletes/' + str(id))
                name = res.json()['displayName']
                return str(name)
            else:

            # Load FAISS index for specified column
                embeddings = OpenAIEmbeddings(
                    model='text-embedding-ada-002'
                )
                db = FAISS.load_local(fr"C:\Users\adith\Documents\Projects\python-projects\CRICAI-SQL\vector_databases\{column_name}", embeddings, allow_dangerous_deserialization=True)

                # Create document retriever and find most relevant document to player name
                retriever = db.as_retriever(search_type='mmr', search_kwargs={'k': 5, 'lambda_mult': 1})
                matched_value = retriever.get_relevant_documents(search_value)
                res=[]
                for i in matched_value:
                    res.append(i.page_content)
                return str(res)

        # print(type(list1))
        # print(list1)


        list_str = re.search(r'\[.*\]', list1)
        if list_str:
            list_str = list_str.group()
        else:
            return "Invalid input format: No list found"
        list1 = ast.literal_eval(list_str)
        # list1 = eval(list_str)
        # list1=eval(list1)

        # print(type(list1))
        # print(list1)
        l=[]
        for list2 in list1:
            if len(list2)!=2:
                print("list2",list2)
                return "Inputs are not in correct format."
                
            res=get_value_from_column1(list2)
            l.append(list2+[res])
        print(l)
        return l 
        
    @tool
    def add_2_numbers(a,b) :
        """ 
        This function adds 2 numbers.
      """
        return a+b
        

    tools = [get_value_from_column,add_2_numbers]

    prompt = PromptTemplate.from_template(
    """
  
    You are an intelligent agent designed to assist in generating SQL queries(specifically filtering dataset i.e in where clause) by processing  based on user query.


    You will be given a user query,in which users may provide vague references to  names, such as batter,bowler,bowler_type,
    batter_hand,bowler_kind,match_type,team_bat,team_bowl,tournament_name,series_name,venue in a cricket database,
    but directly using those names/values of that column for filtering the dataset in sql may not work always.
    -so you need to search for the exact name of that column in the database using the tool get_value_from_column ( except for bowler_kind,batter_hand,match_type which have only few values and i have provided below  you with the list of those values).


    Thus you need  to preprocess these user queries by searching in the database using the tool get_value_from_column and 
    then return the final answer.


    Remember:
    -Even if in the query users mentioned other columns other than the ones mentioned above,you need to ignore them.    




    Important important columns,and its values in the database are:
    columns that has many distinct values are:
    - venue 
    - series_name
    - tournament_name
    - batter
    - bowler
    - team_bat
    - team_bowl

    you may need to search for values in above columns in database.
    info about tournament_name and series_name:

        -tournament_name column  mentions the  general name of series/event but the series_name column
        specifies both the series/event with a season/year (i.e the series_name=tournament_name +season/year).  
        -thus for queries involving just the name of event  can use directly the tournament_name,if specifically
        mentioned the event/tournament with a season/year  can use series_name.




    columns that has few distinct values and its values are known to us are:
    - bowler_kind: ['pace bowler','spin bowler'] 
    - batter_hand :['Right Hand Batter','Left Hand Batter'] 
    - match_type : ['ODI','MDM','Test','IT20','T20','ODM'] 
    - bowler_type : ['RAP'(means  right arm pace),'OB'(means off break),'LWS'(means left wrist spin),
    'LAP'(means left arm pace),'LB'(means left arm break),'SLA'(means slow left arm)
    ] 
    you many need not to search in database for these columns as their values are known to us.




    ---------------------
    You have access to the following tools:

    {tools}
    ---
    Tool names:
    {tool_names}
    ---------------------------------------------------------------------------------------------------

    Use the following format:

    User Query: {input}

    Thought: Analyze the user query and determine what information needs to be retrieved from the database. 
    Consider which columns are relevant and what search terms to use.

    Action: get_value_from_column

    Action Input: [['column_name1', 'search_value1'],['column_name2', 'search_value2'], ...]

    Observation: result of action.


    (Repeat the Thought/Action/Action Input/Observation steps N times, but dont repeat the same action again and again)

    After you have done the above steps N times,

    -understand/reason which values found in database are most relevant and accurate for filtering the dataset for user query. 



    Thought: Analyze the observations from all actions. 


    Final Answer: Provide  Final Answer that includes
    -The relevant column names,appropriate values based on results of actions found for each column. from the database.



    -But Dont include  sql query.



    Begin!

    User Query: {input}
    Thought: {agent_scratchpad}


    """
    )

    chat_history = []
    # prompt = hub.pull("hwchase17/react")
    agent = create_react_agent(llm, tools, prompt)    
    # agent = create_tool_calling_agent(llm, tools, prompt)

    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True,return_intermediate_steps=True)


    result=agent_executor.invoke({"input": user_query})

        # n=404
        # return {"response":str(e),"query":user_query,"f":None}

    print("Time Taken for gpt-4o-mini = ",time.time()-ti)
    print("response of Agent :  ",result['output'])


    
    res_gem+='\n\n'
    return result



In [25]:
result=find_references("klaasen,russel in stats cpl since 2022,")



> Entering new AgentExecutor chain...
Analyze the user query and determine what information needs to be retrieved from the database. The user mentions "klaasen" and "russel", which likely refer to players (batter or bowler), and "cpl" which refers to a tournament. The query also specifies a time frame "since 2022", indicating that we might need to filter based on the relevant tournament name or series name associated with the CPL.

To proceed, I will first check for the values associated with the players "klaasen" and "russel" in the relevant columns (batter and bowler). Then, I will search for the tournament name related to "cpl".

Action: get_value_from_column

Action Input: [['batter', 'klaasen'], ['bowler', 'klaasen'], ['batter', 'russel'], ['bowler', 'russel'], ['tournament_name', 'cpl']]
[['batter', 'klaasen', 'Heinrich Klaasen'], ['bowler', 'klaasen', 'Heinrich Klaasen'], ['batter', 'russel', 'Andre Russell'], ['bowler', 'russel', 'Andre Russell'], ['tournament_name', 'cpl', "

In [40]:
def parse_search_agent_response(result):
    intermediate_output=""
    for step in result['intermediate_steps']:
        intermediate_output += step[0].log + "\n"  + str(step[1]) + "\n"
    string=f"{ result['input']}\n    {intermediate_output}    result['output']\n   "
    return string
parse_search_agent_response(result)



klaasen,russel in stats cpl since 2022,
    Analyze the user query and determine what information needs to be retrieved from the database. The user mentions "klaasen" and "russel", which likely refer to players (batter or bowler), and "cpl" which refers to a tournament. The query also specifies a time frame "since 2022", indicating that we might need to filter based on the relevant tournament name or series name associated with the CPL.

To proceed, I will first check for the values associated with the players "klaasen" and "russel" in the relevant columns (batter and bowler). Then, I will search for the tournament name related to "cpl".

Action: get_value_from_column

Action Input: [['batter', 'klaasen'], ['bowler', 'klaasen'], ['batter', 'russel'], ['bowler', 'russel'], ['tournament_name', 'cpl']]

[['batter', 'klaasen', 'Heinrich Klaasen'], ['bowler', 'klaasen', 'Heinrich Klaasen'], ['batter', 'russel', 'Andre Russell'], ['bowler', 'russel', 'Andre Russell'], ['tournament_name', 'cp

In [24]:
result

{'input': 'klaasen,russel in stats cpl 2022,',
 'output': "- Relevant columns:\n  - batter: ['Heinrich Klaasen', 'Andre Russell']\n  - series_name: ['CPL 2022']",
 'intermediate_steps': [(AgentAction(tool='get_value_from_column', tool_input="[['batter', 'Klaasen'], ['batter', 'Russel'], ['series_name', 'CPL 2022']]\n", log='Analyze the user query and determine what information needs to be retrieved from the database. The user is asking for statistics related to players "Klaasen" and "Russel" in the context of "CPL 2022". Relevant columns for this query would include "batter" for the players\' names and "series_name" for the specific tournament with the season/year.\n\nAction: get_value_from_column\n\nAction Input: [[\'batter\', \'Klaasen\'], [\'batter\', \'Russel\'], [\'series_name\', \'CPL 2022\']]\n'),
   [['batter', 'Klaasen', 'Heinrich Klaasen'],
    ['batter', 'Russel', 'Andre Russell'],
    ['series_name',
     'CPL 2022',
     "['Caribbean Premier League 2022', 'Indian Premier L

In [30]:
result 

{'input': 'klaasen,russel in stats cpl since 2022,',
 'output': '- **Batter**: Heinrich Klaasen, Andre Russell\n- **Bowler**: Heinrich Klaasen, Andre Russell\n- **Tournament Name**: Caribbean Premier League\n- **Series Name**: Caribbean Premier League 2021 (as the most recent series name available) \n\nPlease note that while the series name does not directly indicate seasons from 2022, it may still be relevant for the 2022 stats being queried.',
 'intermediate_steps': [(AgentAction(tool='get_value_from_column', tool_input="[['batter', 'klaasen'], ['bowler', 'klaasen'], ['batter', 'russel'], ['bowler', 'russel'], ['tournament_name', 'cpl']]\n", log='Analyze the user query and determine what information needs to be retrieved from the database. The user mentions "klaasen" and "russel", which likely refer to players (batter or bowler), and "cpl" which refers to a tournament. The query also specifies a time frame "since 2022", indicating that we might need to filter based on the relevant to

In [ ]:

questions = [
    "David Warner shahina Afridi",
    "wickets lost by teams between 7 to 15 over at Chepauk stadium in IPL 2023",
    "Which bowler consider most run against BABAR AZAM",
    "Rohit sharma ICC mens t20 world cup stats",
    "Ruturaj Gaikwad powerplay strike rate in IPL 2024",
    "bowlers against russel,klaasen,stubbs compared to others in last 4 overs in ipl",
    "bumrah in overs 17-20",
    "Narine vs shami",
    "Natrajan to pooran in the ipl",
    "MI score at the end of the powerplay in each match in IPL 2024",
    "Mayank Agarwal vs spin",
    "Virat Kohli stats vs spinners in t20i",
    "Bharat ratna shri adal bihari vajpayee stadium",
    "Shivam dube vs boult",
    "Shakib Al Hasan",
    "Suryakumar yadav 2024 ipl match wise scores",
    "Virat Kohli stats vs spinners in t20i",
    "teams stats  across different phases of games in ipl 2024",
    "sunrisers",
    "batters against narine,rashid,bumrah compared to others",
    "Shaw batting 1st vs batting 2nd at Arun Jaitley stadium in t20",
    "Nitish Kumar reddy",
    "MS Dhoni vs spinners in international",
    "D chameera ipl 2024",
    "Nicolash pooran per balls four against all types spinner bowlers 2023/2024",
    "Virat Kohli vs all bowling types since 2023 in Twenty20",
    "Ravindra jadeja t20.world cup stats year wise",
    "GT last 4 over bowling performance IPL 2024"
]

In [26]:
df

,innings,total_runs,total_balls,total_outs,total_fours,total_sixes,Average,Strike_Rate,hs
0,4,38,39,1,2,1,38.0,97.44,12
